In [1]:
from newscatcher import Newscatcher
import pandas as pd
from datetime import datetime

In [2]:
all_headlines_csv = 'data/google-news-headlines.csv'
all_df_headlines = pd.read_csv(all_headlines_csv)

In [3]:
now = datetime.utcnow().strftime('%Y-%m-%d %H:%M')
now

'2020-02-28 05:06'

In [4]:
# get the current headlines
ns = Newscatcher('news.google.com')
df_latest = pd.DataFrame({'datetime': now, 'headline': ns.headlines})

In [5]:
df_latest

,datetime,headline
0,2020-02-28 05:06,Pence Will Control All Coronavirus Messaging F...
1,2020-02-28 05:06,U.S. workers without protective gear assisted ...
2,2020-02-28 05:06,Coronavirus outbreak: All the latest updates -...
3,2020-02-28 05:06,Milwaukee police identify the 5 victims of Mol...
4,2020-02-28 05:06,You better believe Nancy Pelosi has a Bernie S...
5,2020-02-28 05:06,Trump touts black unemployment at African Amer...
6,2020-02-28 05:06,"Lori Vallow, mother of missing Idaho kids, den..."
7,2020-02-28 05:06,'Bring back the deceased person': police chase...
8,2020-02-28 05:06,NYC progressive groups launch #NeverBloomberg ...
9,2020-02-28 05:06,Massive water main break floods parts of Houst...


In [6]:
# merge with the old one
all_df_headlines = all_df_headlines.append(df_latest, sort=True)


In [7]:
all_df_headlines.drop_duplicates(['headline'], inplace=True)

In [8]:
# save all headlines  to file
all_df_headlines.to_csv(all_headlines_csv, index=False)

In [9]:
# TODO count those with coronavirus